In [ ]:
from gurobipy import *
import numpy as np
import math as mt

In [ ]:
def random_matrice_utilite(n, p):
        return np.random.randint(50 ,size=(n,p))


In [ ]:
def generate_w_prime(n):
        w=[i for i in range(n,0,-1)]
        w1=[]
        for i in range(n):
                if i==n-1 :
                        w1.append(w[i])
                else :
                        w1.append(w[i]-w[i+1])
        return w1

In [ ]:
def generate_rk(n):
        coef_rk = []
        for k in range(n):
                for i in range(n):
                        coef_rk.append([])
                        for j in range(n):
                                coef_rk[-1].append(0)
                        coef_rk[-1][k] = 1
        coef_rk = np.array(coef_rk)
        return coef_rk

In [ ]:
def generate_x(n, p, u):
        inf = 0
        cpt = 0
        coef_xi = []
        for i in range(n):
                coef_xi.append([0 for _ in range(n*p)])
                for j in range(inf, inf+p):
                        coef_xi[-1][j] = - u[i][j%p]
                inf += p
        res = coef_xi
        for i in range(n*n-n):
                res = np.vstack([res, coef_xi[i%n]])
        return res

In [ ]:
def coef_cont_unicite(n, p):
        L2=np.zeros((p,1))
        for i in range(n):
            L1=np.identity(p)
        L2=np.concatenate((L1,L2),axis=1)
        L2=np.delete(L2,[-1],1)
        
        res = L2
        for i in range(n-1):
                res = np.concatenate((res, L2), axis=1)      
        zeros = np.zeros((p, n*n+n))
        return np.concatenate((zeros, res), axis=1)

coef_cont_unicite(3, 2)

In [ ]:
def matrice_contrainte(n, p, u):
        ## les variables rk
        coef_rk = generate_rk(n)
        
        ## les coef de bij 
        coef_bik = -1 * np.identity(n**2)
        
        # les coef de x
        coef_xi = generate_x(n, p, u)
        
        # contrainte sur l'unicite
        deux_personne_pas_le_mm = coef_cont_unicite(n, p)
        
        bik_xi = np.concatenate((coef_bik, coef_xi), axis=1)
        res = np.concatenate(
                (coef_rk, bik_xi), 
                axis=1
        )
        return np.vstack(
                (res, deux_personne_pas_le_mm)
        )

n, p = 3, 4
u = random_matrice_utilite(n, p)
matrice_contrainte(n, p, u)

In [ ]:
def second_membre(p):
        S= [0 for i in range (n*n)]
        for i in range(p):
                S.append(1)
        return S

In [ ]:
def coef_func_objectif(n, p, w_prime):
        ## coefficient fonction objectif
        c = [(i+1) * w_prime[i] for i in range(n)] # coef des r_k
        # coef des b_ik
        for i in range(n) :
                for j in range(n) :
                        c.append(-1*w_prime[i])

        c += [0 for i in range(n*p)] # ajout des coef de x
        c = np.array(c)
        #print(c)
        return c

In [ ]:
# Creer les models de la fonction objectif
m = Model("Experimentation partie 2") 

In [ ]:
def declaration_vars(n, p, m):
        # declaration des variables de decisions
        x = []
        for i in range(n):
                x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="r%d" % (i+1)))
        for i in range (n):
                for j in range(n):
                        x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="b%d%d" % ((j+1,(i+1)))))
        for i in range (n):
                for j in range(p):
                        x.append(m.addVar(vtype=GRB.BINARY, lb=0, name="x%d%d" % ((i+1,(j+1)))))

        # maj du modele pour integrer les nouvelles variables
        m.update()
        return x

In [ ]:
def definition_objectif(n, p, a, x, m):
        # definition de l'objectif
        obj = LinExpr();
        obj =0
        for j in range(n * (n + p +1)):
                obj += a[j] * x[j]

        m.setObjective(obj,GRB.MAXIMIZE)
        return obj

In [ ]:
def definition_contraintes(n, p, m, b, x, S):
        # Definition des contraintes
        lignes = range(n*n+p)
        colonnes = range(n*(n+p+1))

        for i in lignes:
                m.addConstr(quicksum(b[i][j]*x[j] for j in colonnes) <= S[i], "Contrainte%d" % i)

In [ ]:
def display_vars(m):
        print("")                
        print('Solution optimale:')
        mvars = m.getVars()
        names = m.getAttr('VarName', mvars)
        values = m.getAttr('X', mvars)
        result = dict(zip(names, values))
        for key in result:
                print(key,'=',result[key])
        print("")
        print('Valeur de la fonction objectif :', m.objVal)


In [408]:
def experience(n1, p1):
        t=0
        for i in range(10):
                u = random_matrice_utilite(n1, p1)
                w1 = generate_w_prime(n1)
                b = matrice_contrainte(n1, p1, u)
                S = second_membre(p1)
                a = coef_func_objectif(n1, p1, w1)
                # Creer les models de la fonction objectif
                m = Model("Experimentation partie 2")
                m.Params.LogToConsole = 0
                x = declaration_vars(n1, p1, m)
                obj = definition_objectif(n1, p1, a, x, m)
                definition_contraintes(n1, p1, m, b, x, S)
                # Resolution
                m.optimize()
                #display_vars(m)
                t+=m.Runtime
        return t/10
     
ns=[5,10,15,20]
res=[]   
for n in ns:
        p = 5*n
        t=experience(n, p)
        res.append((n, p, t))         

experience(3, 6)


Solution optimale:
r1 = 210.0
r2 = 217.0
r3 = 219.0
r4 = 223.0
r5 = 224.0
r6 = 225.0
r7 = 226.0
r8 = 226.0
r9 = 229.0
r10 = 229.0
b11 = 0.0
b21 = 0.0
b31 = 0.0
b41 = 0.0
b51 = 0.0
b61 = 0.0
b71 = 0.0
b81 = 0.0
b91 = 0.0
b101 = 0.0
b12 = 0.0
b22 = 0.0
b32 = 0.0
b42 = 7.0
b52 = 0.0
b62 = 0.0
b72 = 0.0
b82 = 0.0
b92 = 0.0
b102 = 0.0
b13 = 0.0
b23 = 0.0
b33 = 2.0
b43 = 9.0
b53 = 0.0
b63 = 0.0
b73 = 0.0
b83 = 0.0
b93 = 0.0
b103 = 0.0
b14 = 4.0
b24 = 0.0
b34 = 6.0
b44 = 13.0
b54 = 0.0
b64 = 0.0
b74 = 0.0
b84 = 0.0
b94 = 0.0
b104 = 0.0
b15 = 5.0000000000000036
b25 = 0.0
b35 = 7.0000000000000036
b45 = 14.000000000000004
b55 = 0.0
b65 = 1.0000000000000036
b75 = 0.0
b85 = 0.0
b95 = 0.0
b105 = 0.0
b16 = 6.0
b26 = 0.0
b36 = 8.0
b46 = 15.0
b56 = 0.9999999999999964
b66 = 2.0
b76 = 0.0
b86 = 0.0
b96 = 0.0
b106 = 0.0
b17 = 7.0
b27 = 0.0
b37 = 9.0
b47 = 16.0
b57 = 1.9999999999999964
b67 = 3.0
b77 = 0.0
b87 = 0.0
b97 = 1.0
b107 = 0.0
b18 = 7.0
b28 = 0.0
b38 = 9.0
b48 = 16.0
b58 = 1.9999999999999964
b68

In [ ]:
import matplotlib.pyplot as plt

data=[i[2]for i in res]
p=[5,10,15,20]
c1=[] 
c2=[] 
c3=[]
for i in range (len(data)):
    if i>=0 and i<4:
        c1+=[data[i]]
    elif i>=4 and i<8:
        c2+=[data[i]]
    else:
        c3+=[data[i]]

plt.plot(p,c1, label='n=2')
plt.plot(p,c2,label='n=5')
plt.plot(p,c3,label='n=10')
plt.legend()